In [1]:
from transformers import AutoModel

model = AutoModel.from_pretrained("bert-base-cased")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [4]:
model.save_pretrained("/content/directory_on_my_computer")

In [5]:
ls /content/directory_on_my_computer


config.json  model.safetensors


In [7]:
# To reuse a saved model, use the from_pretrained() method again:
from transformers import AutoModel

model = AutoModel.from_pretrained("/content/directory_on_my_computer")
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(28996, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [8]:
from huggingface_hub import notebook_login

notebook_login()

In [9]:
# Transformer models handle text by turning the inputs into numbers. Here we will look at exactly what happens when your text is
# processed by the tokenizer. We’ve already seen in Chapter 1 that tokenizers split the text into tokens and then convert
# these tokens into numbers. We can see this conversion through a simple tokenizer:



In [12]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

encoded_input = tokenizer("Hello, I'm a single sentence!")
print(encoded_input)

{'input_ids': [101, 8667, 117, 146, 112, 182, 170, 1423, 5650, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [19]:
# You’ll notice that the tokenizer has added special tokens — [CLS] and [SEP] — required by the model.
#  Not all models need special tokens; they’re utilized when a model was pretrained with them,
#  in which case the tokenizer needs to add them as that model expects these tokens.
a=encoded_input["input_ids"]
print(a) #numerical representations of your tokens
b=tokenizer.decode(encoded_input["input_ids"])
print(b)

[101, 8667, 117, 146, 112, 182, 170, 1423, 5650, 106, 102]
[CLS] Hello, I ' m a single sentence! [SEP]


In [21]:
a=encoded_input["token_type_ids"]
print(a) #these tell the model which part of the input is sentence A and which is sentence B
b=b=tokenizer.decode(encoded_input["token_type_ids"])
print(b)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [23]:
a=encoded_input["attention_mask"] #this indicates which tokens should be attended to and which should not
print(a)
b=tokenizer.decode(encoded_input["attention_mask"])
print(b)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1] [unused1]


In [28]:
# You can encode multiple sentences at once, either by batching them together (we’ll discuss this soon) or by passing a list:
encoded_input = tokenizer("How are you?", "I'm fine, thank you!")
print(encoded_input)

{'input_ids': [101, 1731, 1132, 1128, 136, 102, 146, 112, 182, 2503, 117, 6243, 1128, 106, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [30]:
# Note that when passing multiple sentences, the tokenizer returns a list for each sentence for each dictionary value.
# We can also ask the tokenizer to return tensors directly from PyTorch:
encoded_input_tensor = tokenizer("Hello, I'm a single sentence!",return_tensors='pt')

encoded_input_tensor

{'input_ids': tensor([[ 101, 8667,  117,  146,  112,  182,  170, 1423, 5650,  106,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [31]:
# But there’s a problem: the two lists don’t have the same length!
# Arrays and tensors need to be rectangular, so we can’t simply convert these lists to a PyTorch tensor (or NumPy array).
# The tokenizer provides an option for that: padding.





In [41]:
# Padding inputs
# If we ask the tokenizer to pad the inputs, it will make all sentences the same length by adding a
# special padding token to the sentences that are shorter than the longest one:

encoded_input = tokenizer(
    ["How are you?", "I'm fine, thank you!"], padding=True, return_tensors="pt"
)
# print(encoded_input)
print("input_ids:    ",    encoded_input["input_ids"],('\n'))
print("token_type_ids:    ",encoded_input["token_type_ids"],('\n'))
print("attention_mask:    ",encoded_input["attention_mask"],('\n'))


input_ids:     tensor([[ 101, 1731, 1132, 1128,  136,  102,    0,    0,    0,    0],
        [ 101,  146,  112,  182, 2503,  117, 6243, 1128,  106,  102]]) 

token_type_ids:     tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]) 

attention_mask:     tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]) 



In [42]:
# Now we have rectangular tensors! Note that the padding tokens have been encoded into input IDs with ID 0, and
# they have an attention mask value of 0 as well.
# This is because those padding tokens shouldn’t be analyzed by the model: they’re not part of the actual sentence.

In [44]:
# Truncating inputs

In [55]:
# The tensors might get too big to be processed by the model. For instance, BERT was only pretrained with sequences up to 512 tokens,
# so it cannot process longer sequences.
# If you have sequences longer than the model can handle, you’ll need to truncate them with the truncation parameter:

encoded_input = tokenizer(
    "This is a very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very very long sentence.",
    truncation=True,
)
print(encoded_input["input_ids"])

[101, 1188, 1110, 170, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1263, 5650, 119, 102]


In [48]:
a=[101, 1188, 1110, 170, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304,
 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304,
 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304,
 1304, 1304, 1304, 1304, 1304, 1263, 5650, 119, 102]



In [49]:
len(a)

57

In [56]:
b=[101, 1188, 1110, 170, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1304, 1263, 5650, 119, 102]


len(b)


57

In [58]:
encoded_input = tokenizer(
    ["How are you?", "I'm fine, thank you!"],
    padding=True,
    truncation=True,
    max_length=5,
    return_tensors="pt",
)
# print(encoded_input)
print("input_ids:    ",    encoded_input["input_ids"],('\n'))
print("token_type_ids:    ",encoded_input["token_type_ids"],('\n'))
print("attention_mask:    ",encoded_input["attention_mask"],('\n'))

input_ids:     tensor([[ 101, 1731, 1132, 1128,  102],
        [ 101,  146,  112,  182,  102]]) 

token_type_ids:     tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]) 

attention_mask:     tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]]) 



In [59]:
# # Adding special tokens
# Special tokens (or at least the concept of them) is particularly important to BERT and derived models.
# These tokens are added to better represent the sentence boundaries,
# such as the beginning of a sentence ([CLS]) or separator between sentences ([SEP]). Let’s look at a simple example:

In [61]:
encoded_input = tokenizer("How are you?")
print(encoded_input["input_ids"])
tokenizer.decode(encoded_input["input_ids"])

[101, 1731, 1132, 1128, 136, 102]


'[CLS] How are you? [SEP]'

In [79]:
#-------------- Why is all of this necessary?   🧪 Goal: Feed raw text to a model and get the output-------------------------------------------

sequences = [
    "I've been waiting for a HuggingFace course my whole life.",
    "I hate this so much!",
]

In [80]:
# Step 1:
from transformers import AutoTokenizer, AutoModel
import torch


In [83]:
#Step 2: Load tokenizer and model
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [86]:
# Step 3: Tokenize the input sentence

# 101: special [CLS] token

# 102: special [SEP] token

# Numbers in between are token IDs from tokenizer’s vocabulary.

# attention_mask tells the model which tokens to focus on (1) or ignore (0 for padding).


sentence = "I hate this so much!"
encoded_input = tokenizer(sentence, return_tensors="pt")
print(encoded_input)

{'input_ids': tensor([[ 101, 1045, 5223, 2023, 2061, 2172,  999,  102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1]])}


In [87]:
# Step 4: Feed tokenized input to the model
with torch.no_grad():  # no need to compute gradients for inference
    outputs = model(**encoded_input)

In [90]:
# Step 5: Check the output
print(outputs.last_hidden_state.shape)     #outputs.last_hidden_state is a matrix of shape [batch_size, sequence_length, hidden_size]



torch.Size([1, 8, 768])
